In [1]:
import merge

# Results

In [2]:
merge.team_names()

['b', 'c_big', 'c_old']

In [130]:
merge.validate_result('c_big')

,orderID,articleID,colorCode,sizeCode,confidence,prediction
0,1744314,1000277,7142,27,0.740000,0.0
1,1744318,1000277,7142,27,0.605902,0.0
2,1760100,1000277,7142,27,0.630000,0.0
3,1775450,1000277,7142,27,NaN,0.0
4,1792975,1000277,7142,27,0.700000,0.0
5,1811492,1000277,7142,27,0.700000,1.0
6,1823428,1000277,7142,27,0.570000,1.0
7,1843207,1000277,7142,27,0.550000,0.0
8,1745688,1000296,2088,44,0.600000,0.0
9,1751108,1000296,2081,44,0.527165,1.0


## Mean accuracies

In [131]:
accuracies = merge.mean_accuracies()
accuracies

TypeError: type object argument after * must be a sequence, not map

# Split performance

In [117]:
merge.evaluate()

TypeError: type object argument after * must be a sequence, not map

In [115]:
merge.merge()

KeyError: 'prediction'

# Merge predictions

## Weighted Majority Vote

We use the following formula for the final $prediction_i$, with $i$ being the row index. In it, each $prediction_{c,i}$ of classifier $c$ is weighted.

$$
row_i = 
\dfrac
    {\sum_{c \in C} 
        prediction_{c,i} 
        \cdot weight_{c,i}} 
    {\sum_{c \in C} weight_{c,i}}
$$

The $weight_{c,i}$ takes into account the condfidence of $c$ in row $i$, the mean confidence of $c$ over all rows, the overall accuracy of $c$ and the mean accuracy of all classifiers.

$$
weight_{c,i} = \dfrac{confidence_{c,i}}{confidence_{c,\varnothing}} \cdot \bigg(\dfrac{accuracy_c}{accuracy_\varnothing}\bigg)^2
$$

The confidences returned by the classifiers are, as such, not comparable to each other. They differ in meaning and in range. To counteract and to avoid favoring classifiers that are generally -maybe mistakenly- confident, we take the confidence compared to the classifier's mean confidence.

The accuracy, we believe, is the best predictor of a classifier's performance in the target set. We take this information from a test set that each group used to evaluate their classifier. The test set is a 70/30 split of the training data. We compare the accuracy to the mean accuracy of all classifiers. As the difference tends to be big in information, but small in extent, the ratio is squared.

In [ ]:
merge.majority_vote(merge.merge(team_results), accuracies)

# Evaluate team

In [221]:
splits = merge.evaluate_team('cd')